In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import math
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/student-performance/StudentPerformance.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
# Encoded Salah Satu Kolom
le = LabelEncoder()
df['Extracurricular Activities'] = le.fit_transform(df['Extracurricular Activities'])
df.head()

In [ ]:
sns.boxplot(data=df)
plt.xticks(rotation=45)
plt.show

In [ ]:
cols = df.columns # panggil semua kolom

fig, axes = plt.subplots(2, 3, figsize=(15, 5)) # buat kanvas (fig) dan tentukan banyak frame/gambar
# (axes)
axes = axes.flatten() # ukuran 2, 3 tadi menjadikan arraynya berukuran 2D, maka harus dibuat kembali
# menjadi 1D dengan .flatten()

for ax, col in zip(axes, cols):
    sns.histplot(data=df, x=col, kde=True, ax=ax)
    ax.set_title(col)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
corr_mtx = df.corr()
sns.heatmap(
    corr_mtx,
    annot=True,
    cmap='coolwarm',
    linewidths=0.5
)
plt.xticks(rotation=45)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Performance Index', axis=1)
y = df['Performance Index']
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

print(f"Ukuran X_train: {X_train.shape}")
print(f"Ukuran X_test: {X_test.shape}")

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression() # panggil model
model.fit(X_train_scaled, y_train) # isi bahan ke model

y_pred = model.predict(X_test_scaled) # langsung uji

score = r2_score(y_test, y_pred) # cek errornya
mse = mean_squared_error(y_test, y_pred) # # cek errornya
rmse = np.sqrt(mse)

print(f"R-Squared: {score:.4f}")
print(f"RMSE: {rmse:.4f}")

In [ ]:
coef = pd.DataFrame({
    "Fitur": X_train.columns,
    "Coef": model.coef_
})
print(coef)

In [ ]:
import joblib
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(model, "linear_regression_model.pkl")
joblib.dump(list(X_train.columns), "features.pkl")
metrics = {
    "r2": score,
    "rmse": rmse
}
joblib.dump(metrics, "metrics.pkl")

**TES**

In [ ]:
model = joblib.load("linear_regression_model.pkl")
scaler = joblib.load("scaler.pkl")
features = joblib.load("features.pkl")

In [ ]:
new_df = pd.DataFrame({
    "Hours Studied": [15],
    "Sleep Hours": [7],
    "Previous Scores": [98],
    "Extracurricular Activities": [1],
    "Sample Question Papers Practiced": [10],
})
new_df = new_df[features]

In [ ]:
new_df_scaled = scaler.transform(new_df)
y_predd = model.predict(new_df_scaled)
print(f"Prediksi Performance Score: {y_predd}")